Web Crawler

In [ ]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element("xpath",'.//span').text
    try:
        handle = card.find_element("xpath",'.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return

    try:
        postdate = card.find_element("xpath",'.//time').get_attribute('datetime')
    except NoSuchElementException:
        return

    comment = card.find_element("xpath",'.//div[2]/div[2]/div[1]').text
    responding = card.find_element("xpath",'.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element("xpath",'.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element("xpath",'.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element("xpath",'.//div[@data-testid="like"]').text

    # get a string of all emojis contained in the tweet
    """Emojis are stored as images... so I convert the filename, which is stored as unicode, into
    the emoji character."""
    emoji_tags = card.find_elements("xpath",'.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)

    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt)
    return tweet

In [ ]:
# application variables
user = input('username: ')
my_password = getpass('Password: ')
search_term = input('search term: ')


# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()
sleep(5)
username = driver.find_element("xpath",'//input[@name="text"]')
username.send_keys(user)
username.send_keys(Keys.RETURN)
sleep(3)

password = driver.find_element('xpath','//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(3)

# find search input and search for term
search_input = driver.find_element('xpath','//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element("link text",'Latest').click()

username: VateamK
Password: ········
search term: #CSKvsRR


C:\Users\kldat\AppData\Local\Temp\ipykernel_16340\2738353507.py:10: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [ ]:
# get all tweets on the page
data = []
twitter_tweet_ = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements('xpath', '//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
                twitter_tweet_.append(tweet[3])

    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [ ]:
with open('ProjectK.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Emojis', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [ ]:
import pandas as pd
header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Emojis', 'Comments', 'Likes', 'Retweets']
df = pd.DataFrame(data,columns= header)
df.head()

,UserName,Handle,Timestamp,Text,Emojis,Comments,Likes,Retweets
0,_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ,@SaiPrav26957591,2023-04-13T07:51:05.000Z,"_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ\n@SaiPrav26957591\n·\n5m""KARMA"" ...",🇵 🇷 🇦 🇻 🇪 🇪 🇳 😊 😊 🤩,,,
1,Rozana Spokesman,@RozanaSpokesman,2023-04-13T07:50:34.000Z,Rozana Spokesman\n@RozanaSpokesman\n·\n6m#IPL2...,,,,
2,Broadcast Media,@vijendra_deepa,2023-04-13T07:50:11.000Z,Broadcast Media\n@vijendra_deepa\n·\n6mThe uni...,,,,
3,Venky_K,@VenkyK_Offic,2023-04-13T07:49:37.000Z,Venky_K\n@VenkyK_Offic\n·\n6mBeautiful Video.....,,,,
4,Albin Prince,@prinz_albin,2023-04-13T07:48:57.000Z,Albin Prince\n@prinz_albin\n·\n7mMy Man \n@msd...,😭 💛 🥹 🛐 🔥 🧎 💥 ❤ 💯,,,2


In [ ]:
twitter_tweet = pd.array(twitter_tweet_)
df['Text'] = twitter_tweet

In [ ]:
tweet

('Ruchita Jain #savesoil',
 '@Ruchitaamehtaa',
 '2023-04-13T06:54:25.000Z',
 'Ruchita Jain #savesoil\n@Ruchitaamehtaa\n·\n1hNamaskaram #MSDhoni #ChennaiSuperKings #CSKvsRR #ishafoundation #ishayoga #sadhguru #IshaFoundation #Sadhguru \n@ishafoundation',
 '🙏 🌸 🙌',
 '',
 '',
 '1')

In [ ]:
twitter_tweet

<StringArray>
[                                                                                                                                                                    '_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ\n@SaiPrav26957591\n·\n5m"KARMA" is Boomerang  \n\nCSGay fans crying is my source of happiness.  Their tears work as glucose for my body, their screams bring peace to my ears. \n\nBest Scenery of this IPL \n\n#CSKvsRR',
                                                                                                                                                                                                                                                                        'Rozana Spokesman\n@RozanaSpokesman\n·\n6m#IPL2023: #RajasthanRoyals beat #ChennaiSuperKings by 3 runs \n\n#CSKvsRR #RRvsCSK #IPLT20',
                                                                                                          "Broadcast Media\n@vijendra_deepa\n·\n6mThe unit of Actress \n@aishu_dil\n's laugh

In [ ]:
import pandas as pd
# Read the CSV file into a DataFrame
MYdata = pd.read_csv('ProjectK.csv')

# Print the DataFrame
MYdata.head()

,UserName,Handle,Timestamp,Text,Emojis,Comments,Likes,Retweets
0,_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ,@SaiPrav26957591,2023-04-13T07:51:05.000Z,"_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ\n@SaiPrav26957591\n·\n5m""KARMA"" ...",🇵 🇷 🇦 🇻 🇪 🇪 🇳 😊 😊 🤩,NaN,NaN,NaN
1,Rozana Spokesman,@RozanaSpokesman,2023-04-13T07:50:34.000Z,Rozana Spokesman\n@RozanaSpokesman\n·\n6m#IPL2...,NaN,NaN,NaN,NaN
2,Broadcast Media,@vijendra_deepa,2023-04-13T07:50:11.000Z,Broadcast Media\n@vijendra_deepa\n·\n6mThe uni...,NaN,NaN,NaN,NaN
3,Venky_K,@VenkyK_Offic,2023-04-13T07:49:37.000Z,Venky_K\n@VenkyK_Offic\n·\n6mBeautiful Video.....,NaN,NaN,NaN,NaN
4,Albin Prince,@prinz_albin,2023-04-13T07:48:57.000Z,Albin Prince\n@prinz_albin\n·\n7mMy Man \n@msd...,😭 💛 🥹 🛐 🔥 🧎 💥 ❤ 💯,NaN,NaN,2.0


In [ ]:
MYdata['Text'] = twitter_tweet
MYdata.head()

,UserName,Handle,Timestamp,Text,Emojis,Comments,Likes,Retweets
0,_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ,@SaiPrav26957591,2023-04-13T07:51:05.000Z,"_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ @SaiPrav26957591 · 5m""KARMA"" is ...",🇵 🇷 🇦 🇻 🇪 🇪 🇳 😊 😊 🤩,NaN,NaN,NaN
1,Rozana Spokesman,@RozanaSpokesman,2023-04-13T07:50:34.000Z,Rozana Spokesman @RozanaSpokesman · 6m#IPL2023...,NaN,NaN,NaN,NaN
2,Broadcast Media,@vijendra_deepa,2023-04-13T07:50:11.000Z,Broadcast Media @vijendra_deepa · 6mThe unit o...,NaN,NaN,NaN,NaN
3,Venky_K,@VenkyK_Offic,2023-04-13T07:49:37.000Z,Venky_K @VenkyK_Offic · 6mBeautiful Video.. Da...,NaN,NaN,NaN,NaN
4,Albin Prince,@prinz_albin,2023-04-13T07:48:57.000Z,Albin Prince @prinz_albin · 7mMy Man @msdhoni...,😭 💛 🥹 🛐 🔥 🧎 💥 ❤ 💯,NaN,NaN,2.0


Natural Language Processing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import pandas as pd
df = pd.read_csv(r'C:\Users\kldat\Downloads\spam_ham.csv', encoding="ISO-8859-1").iloc[:, :2]

In [ ]:
df = df.rename(columns={'v1': 'Type', 'v2': 'Tweet'})

In [ ]:
df.head()

,Tweet,Type
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


Remove Punctuations

In [ ]:
# Function to remove punctuations.
def remove_punc(text):
    nonP_text = "".join([char for char in text if char not in string.punctuation])
    return nonP_text

df["body_text_clean"] = df["Tweet"].apply(lambda x: remove_punc(x))

df.head()

,Tweet,Type,body_text_clean
0,"Go until jurong point, crazy.. Available only ...",ham,Go until jurong point crazy Available only in ...
1,Ok lar... Joking wif u oni...,ham,Ok lar Joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...,ham,U dun say so early hor U c already then say
4,"Nah I don't think he goes to usf, he lives aro...",ham,Nah I dont think he goes to usf he lives aroun...


In [ ]:
import re

#function to apply tokenization
def tokenize(text):
    tokens = re.split("\W+", text)# W+ means all capital, small alphabets and integers 0-9
    return tokens

df["body_text_tokenized"] = df["body_text_clean"].apply(lambda x: tokenize(x))

df.head()

,Tweet,Type,body_text_clean,body_text_tokenized
0,"Go until jurong point, crazy.. Available only ...",ham,Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o..."
1,Ok lar... Joking wif u oni...,ham,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]"
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,U dun say so early hor... U c already then say...,ham,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t..."
4,"Nah I don't think he goes to usf, he lives aro...",ham,Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l..."


In [ ]:
#!pip install nltk

Remove Stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kldat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words("english")

def remove_stopwords(token):
    text = [word for word in token if word not in stopwords]# to remove all stopwords
    return text

df["body_text_nonstop"] = df["body_text_tokenized"].apply(lambda x: remove_stopwords(x))
df.head()

,Tweet,Type,body_text_clean,body_text_tokenized,body_text_nonstop
0,"Go until jurong point, crazy.. Available only ...",ham,Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n..."
1,Ok lar... Joking wif u oni...,ham,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]"
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,U dun say so early hor... U c already then say...,ham,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]"
4,"Nah I don't think he goes to usf, he lives aro...",ham,Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around..."


Stemming

In [ ]:
ps = nltk.PorterStemmer()

def stemming(t_text):
    text = [ps.stem(word) for word in t_text]
    return text

df["body_text_stemmed"] = df["body_text_nonstop"].apply(lambda x: stemming(x))
df.head()

,Tweet,Type,body_text_clean,body_text_tokenized,body_text_nonstop,body_text_stemmed
0,"Go until jurong point, crazy.. Available only ...",ham,Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,Ok lar... Joking wif u oni...,ham,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,U dun say so early hor... U c already then say...,ham,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]","[u, dun, say, earli, hor, u, c, alreadi, say]"
4,"Nah I don't think he goes to usf, he lives aro...",ham,Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around...","[nah, i, dont, think, goe, usf, live, around, ..."


Lemmatization

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kldat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kldat\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(t_text):
    text = [wn.lemmatize(word) for word in t_text]
    return text

df["body_text_lemmatized"] = df["body_text_stemmed"].apply(lambda x: lemmatizer(x))
df.head()

,Tweet,Type,body_text_clean,body_text_tokenized,body_text_nonstop,body_text_stemmed,body_text_lemmatized
0,"Go until jurong point, crazy.. Available only ...",ham,Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,Ok lar... Joking wif u oni...,ham,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,U dun say so early hor... U c already then say...,ham,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]","[u, dun, say, earli, hor, u, c, alreadi, say]","[u, dun, say, earli, hor, u, c, alreadi, say]"
4,"Nah I don't think he goes to usf, he lives aro...",ham,Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around...","[nah, i, dont, think, goe, usf, live, around, ...","[nah, i, dont, think, goe, usf, live, around, ..."


In [ ]:
df.to_csv("pre_processed_data.csv", sep=',')

In [ ]:
import pandas as pd
import re
import string
import nltk

data = pd.read_csv("pre_processed_data.csv", sep=',')

Apply Count Vectorizer

1. Convert the text into lower case
2. Remove punctuations
3. Split the text into tokens
4. Remove all stop words
5. Stems each word in the text using the porter stemmer algorithm

In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

1. We are using Count vectorizer to transform the text data into a matrix of token counts.
2. The function, get_feature_names_out method return the list of feature names(tokens) in the order they appear in the matrix. These feature names represent the vocabolary of the CountVectorizer model from the input data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(analyzer=clean_text)
x_count = count.fit_transform(data["Tweet"])

In [ ]:
x_count

<20469x37328 sparse matrix of type '<class 'numpy.int64'>'
	with 174781 stored elements in Compressed Sparse Row format>

In [ ]:
x_count_df = pd.DataFrame(x_count.toarray(), columns=count.get_feature_names_out())

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = data.body_text_clean
y = data.Type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16375,)
(4094,)
(16375,)
(4094,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer()

In [ ]:
# learn training data vocabulary, then use it to create a document-term matrix
X_train_dtm = vect.transform(X_train)

# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

# examine the document-term matrix
X_train_dtm

<16375x36684 sparse matrix of type '<class 'numpy.int64'>'
	with 192966 stored elements in Compressed Sparse Row format>

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<4094x36684 sparse matrix of type '<class 'numpy.int64'>'
	with 41214 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)

<16375x36684 sparse matrix of type '<class 'numpy.float64'>'
	with 192966 stored elements in Compressed Sparse Row format>

Machine Learning Models

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
accuracy = accuracy_score(y_test, y_pred)
print('Multinomial Naives Theorem Accuracy:', accuracy)

Multinomial Naives Theorem Accuracy: 0.82266731802638


In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
accuracy = accuracy_score(y_test, y_pred)
print('SVM Accuracy:', accuracy)


SVM Accuracy: 0.8041035661944309


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
accuracy = accuracy_score(y_test, y_pred)
print('Gradient Boosting Accuracy:', accuracy)


Gradient Boosting Accuracy: 0.7484123106985833


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=10000)
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
accuracy = accuracy_score(y_test, y_pred)
print('Logistic Regression Accuracy:', accuracy)


Logistic Regression Accuracy: 0.8243771372740596


In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
accuracy = accuracy_score(y_test, y_pred)
print('Decision Tree Accuracy:', accuracy)


Decision Tree Accuracy: 0.7725940400586224


Training

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_train_dtm)
accuracy = accuracy_score(y_train, y_pred)
print('Multinomial Naives Theorem Accuracy:', accuracy)

Multinomial Naives Theorem Accuracy: 0.9229312977099237


In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_train_dtm)
accuracy = accuracy_score(y_train, y_pred)
print('SVM Accuracy:', accuracy)


SVM Accuracy: 0.9483969465648855


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_train_dtm)
accuracy = accuracy_score(y_train, y_pred)
print('Gradient Boosting Accuracy:', accuracy)


Gradient Boosting Accuracy: 0.7660458015267175


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=10000)
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_train_dtm)
accuracy = accuracy_score(y_train, y_pred)
print('Logistic Regression Accuracy:', accuracy)


Logistic Regression Accuracy: 0.9865648854961832


In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_train_dtm)
accuracy = accuracy_score(y_train, y_pred)
print('Decision Tree Accuracy:', accuracy)


Decision Tree Accuracy: 0.9999389312977099


In [ ]:
twitter_tweet

<StringArray>
[                                                                                                                                                                    '_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ\n@SaiPrav26957591\n·\n5m"KARMA" is Boomerang  \n\nCSGay fans crying is my source of happiness.  Their tears work as glucose for my body, their screams bring peace to my ears. \n\nBest Scenery of this IPL \n\n#CSKvsRR',
                                                                                                                                                                                                                                                                        'Rozana Spokesman\n@RozanaSpokesman\n·\n6m#IPL2023: #RajasthanRoyals beat #ChennaiSuperKings by 3 runs \n\n#CSKvsRR #RRvsCSK #IPLT20',
                                                                                                          "Broadcast Media\n@vijendra_deepa\n·\n6mThe unit of Actress \n@aishu_dil\n's laugh

My_Data

In [ ]:
MYdata

,UserName,Handle,Timestamp,Text,Emojis,Comments,Likes,Retweets
0,_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ,@SaiPrav26957591,2023-04-13T07:51:05.000Z,"_𝗡𝗧𝗥 ʳᶜᵇᶜᵘˡᵗˢ @SaiPrav26957591 · 5m""KARMA"" is ...",🇵 🇷 🇦 🇻 🇪 🇪 🇳 😊 😊 🤩,NaN,NaN,NaN
1,Rozana Spokesman,@RozanaSpokesman,2023-04-13T07:50:34.000Z,Rozana Spokesman @RozanaSpokesman · 6m#IPL2023...,NaN,NaN,NaN,NaN
2,Broadcast Media,@vijendra_deepa,2023-04-13T07:50:11.000Z,Broadcast Media @vijendra_deepa · 6mThe unit o...,NaN,NaN,NaN,NaN
3,Venky_K,@VenkyK_Offic,2023-04-13T07:49:37.000Z,Venky_K @VenkyK_Offic · 6mBeautiful Video.. Da...,NaN,NaN,NaN,NaN
4,Albin Prince,@prinz_albin,2023-04-13T07:48:57.000Z,Albin Prince @prinz_albin · 7mMy Man @msdhoni...,😭 💛 🥹 🛐 🔥 🧎 💥 ❤ 💯,NaN,NaN,2.0
5,Arman Malik,@Armanmalik9582,2023-04-13T07:48:32.000Z,Arman Malik @Armanmalik9582 · 8mMost Wins vs C...,NaN,NaN,NaN,NaN
6,Monu Bhagwat,@imonubhagwat,2023-04-13T07:46:40.000Z,Monu Bhagwat @imonubhagwat · 9mMahendra singh ...,🤩,NaN,NaN,NaN
7,HT Sports,@HTSportsNews,2023-04-13T07:39:02.000Z,HT Sports @HTSportsNews · 18mR Ashwin reacted ...,NaN,NaN,1.0,6.0
8,Kanak News,@kanak_news,2023-04-13T07:38:23.000Z,Kanak News @kanak_news · 18mନିଲାମରେ ଥିଲେ ଅନସୋଲ...,NaN,NaN,NaN,14.0
9,Jashpal Machhar,@jashpal_machhar,2023-04-13T07:38:10.000Z,Jashpal Machhar @jashpal_machhar · 18mMahendra...,🔥 🔥,NaN,NaN,NaN


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(analyzer=clean_text)
x_count1 = count.fit_transform(MYdata["Text"])

In [ ]:
x_count_df1 = pd.DataFrame(x_count1.toarray(), columns=count.get_feature_names_out())

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(x_count_df1)
X_test_dtm

<619x36684 sparse matrix of type '<class 'numpy.int64'>'
	with 172 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=10000)
clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
y_pred

array(['ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham',